# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
# This filter is as naive as possible and is adapted to be
# computed in integer domain to improve performance.
# The idea is: old data has higher priority then new one
# when averaging them.
class NaiveBayesianFilter():
    def __init__(self, coeff = 2):
        assert(coeff >= 2)
        self.coeff = coeff
        self.data = None
    
    def update(self, new_data):
        if self.data is None:
            self.data = new_data
            return self.data
        old_data = self.data * (self.coeff - 1)
        self.data = (old_data + new_data) // self.coeff
        return self.data
    
    def reset(self):
        self.data = None
        
# The filter coefficient has to be adjusted for a particular purpose.
# Based on the test videos I have selected 8.
bayesian_filter = NaiveBayesianFilter(8)
       


In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
        
def get_average_left_and_right_lanes(lines, image_height, image_width):
    """
    This function extracts 2 averaged 'left' and 'right' lane lines form
    an input array of lines, found by HoughLines transform.
    Expects 'lines' to be an array of [x1, y1, x2, y2] records.
    Returns a list of 2 lines (left and right), 4 coordinates per each.
    """
    # Note: Here and after a 'slope' and an 'offset' mean members of a line equation
    # "y = slope * x + offset"
    left_lines_total = 0
    left_slope_sum = 0.
    left_offset_sum = 0
    right_lines_total = 0.
    right_slope_sum = 0
    right_offset_sum = 0
    
    # While the car's camera remains close to the lane center and the lane is
    # either straight or has a large radius, both left and right lane lines:
    # 1) must have close (but inverted) slope values
    # 2) the absolute slope value should lie within a particular range -- 
    # neither vertical, nor horizontal lines can not be the one we look for. 
    # Experimentally adjusted range for the test videos is:
    min_slope = 0.40
    max_slope = 0.85
    
    # To make lines' top endings fancier, I set up a top limit deliberately:
    roi_top = image_height * 64 // 100
    
    for i in range (0, len(lines)):
        (x1, y1, x2, y2) = lines[i][0]

        # Note: As the image has an inverted Y axis, all the classic line geometry
        # formulas must use '(image_height - y)' instead of 'y'.

        slope = ((image_height - y2) - (image_height - y1)) / (x1 - x2)
        if (abs(slope) < min_slope) or (abs(slope) > max_slope):
            continue
            
        offset = (x2 * y1 - (x2 - x1) * image_height - x1 * y2) / (x1 - x2)
                
        # A 'left' line would have a negative slope, while a 'right' one -- positive.
        if slope > 0.:
            right_slope_sum += slope
            right_lines_total += 1
            right_offset_sum += offset
        else:
            left_slope_sum += slope
            left_lines_total += 1
            left_offset_sum += offset
            
    # When no corresponding line is detected,
    # we also must protect from a division by zero.
    left_avg_slope = left_slope_sum / max(left_lines_total, 1)
    right_avg_slope = right_slope_sum / max(right_lines_total, 1)
    left_avg_offset = left_offset_sum / max(left_lines_total, 1)
    right_avg_offset = right_offset_sum / max(right_lines_total, 1)
    
    # Now, having both lane lines equations with averaged parameters,
    # we only need to compute 2 points per a line for neat displaying.
    # Both bottom points have 'y' equal to image bottom, and both top points
    # have y equal to a ROI top coordinate; so, instead of a classic 
    # "y = kx + b --> x=(b - y)/k", equations for 'x' coord become simpler:
    if left_avg_slope < 0:
        left_bottom_x = int(left_avg_offset / left_avg_slope)
        left_top_x = int((left_avg_offset - (image_height - roi_top)) / left_avg_slope)
    else:
        # In case of failed detection, draw a line on the left image border.
        left_bottom_x = 0; left_top_x = 0

    if right_avg_slope > 0:
        right_bottom_x = int(right_avg_offset / right_avg_slope)
        right_top_x = int((right_avg_offset - (image_height - roi_top)) / right_avg_slope)
    else:
        # In case of failed detection, draw a line on the right image border.
        right_bottom_x = image_width; right_top_x = image_width
    
    lane_lines = np.array([[right_top_x, roi_top, right_bottom_x, image_height],
                           [left_top_x, roi_top, left_bottom_x, image_height]])
    return lane_lines       

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    img_height = img.shape[0]; img_width = img.shape[1]
    lane_lines = get_average_left_and_right_lanes(lines, img_height, img_width)
    lane_lines = bayesian_filter.update(lane_lines)
    for line in [lane_lines]:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img,
                            rho,
                            theta,
                            threshold,
                            np.array([]),
                            minLineLength=min_line_len,
                            maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)
        
    

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
if not os.path.isdir("test_images_output/"):
    os.mkdir("test_images_output")
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
def to_radian(deg):
    """ A helper function to set up the theta for hough lines """
    return deg * np.pi / 180.0

def get_roi_coordinates(image_):
    """
    A helper builder for a ROI points array; a ROI for lane lines detection
    is always a trapezoid, so it is convenient to fix bottom-left and bottom-right
    points to the image bottom corners (with a minor offset) respectively,
    and only move the top part of the trapezoid, changing also its width.
    As the horizon is mostly at the vertical center of the image, the
    trapezoid top coordinate might be set as a vertical offset from the
    vertical center.
    """
    top_halfwidth = 20
    height_offset = 50
    shape = image_.shape
    (height, width) = shape[0:2]
    hor_center = width // 2
    ver_center = height // 2

    bl = (int(width * 0.05), height)
    br = (int(width * 0.95) , height)
    tl = (hor_center - top_halfwidth, ver_center + height_offset)
    tr = (hor_center + top_halfwidth, ver_center + height_offset)
    return np.array([[bl, tl, tr, br]])

In [ ]:
# Some tuneable parameters for lane detection functions
canny_low = 80
canny_high = 200
blur_radius = 3
rho = 2               # distance resolution in pixels of the Hough grid
theta = to_radian(3)  # angular resolution in radians of the Hough grid
threshold = 40        # minimum number of votes (intersections in Hough grid cell)
min_line_length = 30  # minimum number of pixels making up a line
max_line_gap = 200    # maximum gap in pixels between connectable line segments

def preprocess(image_):
    """ Does some preprocessing of an input image/frame """
    luminosity_channel_idx = 1; saturation_channel_idx = 2
    # A convertion to [Hue, Luminocity, Saturation] color space provides
    # the maximum information about lane lines, as they must be always much 
    # brighter then their background, despite their color. Therefore a sum
    # of luminocity and saturation (despite the hue) is what we need.
    # Note: Whether an input is BGR or RGB does not matter here,
    # as luminosity and saturation would be exactly the same.
    hls = cv2.cvtColor(image_, cv2.COLOR_BGR2HLS)
    lum = hls[:,:,luminosity_channel_idx]
    cv2.imwrite('illustrations/luminosity.jpg', cv2.resize(lum, (480,270)))
    sat = hls[:,:,saturation_channel_idx]
    cv2.imwrite('illustrations/saturation.jpg', cv2.resize(sat, (480,270)))
    # A 50/50 sum of lum and sat works good for 'challenge' video, where
    # the road backgorund color changes rapidly and the lane marking is 
    # a bleached yellow. For the rest of videos a 'lum' channel only is enough.
    gray = (lum // 2) + (sat // 2)
    cv2.imwrite('illustrations/gray.jpg', cv2.resize(gray, (480,270)))
    truegray = cv2.cvtColor(image_, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('illustrations/truegray.jpg', cv2.resize(truegray, (480,270)))
    blurred = gaussian_blur(gray, blur_radius)
    cv2.imwrite('illustrations/blurred.jpg', cv2.resize(blurred, (480,270)))
    edges = canny(blurred, canny_low, canny_high)
    cv2.imwrite('illustrations/edges.jpg', cv2.resize(edges, (480,270)))
    masked = region_of_interest(edges, get_roi_coordinates(gray))
    cv2.imwrite('illustrations/masked.jpg', cv2.resize(masked, (480,270)))
    return masked

def find_lanes_on(image_):
    masked = preprocess(image_)
    lane_lines = hough_lines(masked,
                             rho,
                             theta,
                             threshold,
                             min_line_length,
                             max_line_gap)
    cv2.imwrite('illustrations/lane_lines.jpg', cv2.resize(lane_lines, (480,270)))
    result = weighted_img(lane_lines, image_)
    return result


for test_img_filename in os.listdir("test_images/"):
    out_image = find_lanes_on(mpimg.imread('test_images/' + test_img_filename))
    cv2.imwrite('test_images_output/' + test_img_filename,
                cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB))
    bayesian_filter.reset()

    

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    result = find_lanes_on(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'

clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
bayesian_filter.reset()
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
bayesian_filter.reset()
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
bayesian_filter.reset()
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))